In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
! pip install glove-python-binary

     |████████████████████████████████| 948 kB 7.3 MB/s 


In [5]:
from glove import Glove
glove=Glove()
glove_model=glove.load("/content/drive/MyDrive/FYP/IMDb/Glove/glove.model")

In [6]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/IMDb/IMDB-preprocessed.csv")
y=df["sentiment"]


In [7]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [8]:

import tensorflow as tf
maxlen=66
max_words=len(glove_model.dictionary)
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [9]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in glove_model.dictionary:
    embed_vector=glove_model.word_vectors[glove_model.dictionary[word]]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

In [10]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.08644114, -0.46788544,  0.19239497, ...,  0.43217102,
        -0.3802913 ,  0.18661583],
       [ 0.03671082, -0.37766596, -0.04600333, ...,  0.34606747,
        -0.33929718, -0.09252944],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense,Flatten
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=66,embeddings_initializer=Constant(embed_matrix)))
model.add(Flatten()) # loss stucks at about 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(1,activation='sigmoid'))

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)

Epoch 1/100
586/586 [==============================] - 18s 25ms/step - loss: 0.6970 - accuracy: 0.5030
Epoch 2/100
586/586 [==============================] - 14s 25ms/step - loss: 0.6932 - accuracy: 0.5024
Epoch 3/100
586/586 [==============================] - 14s 25ms/step - loss: 0.6932 - accuracy: 0.4969
Epoch 4/100
586/586 [==============================] - 14s 25ms/step - loss: 0.6932 - accuracy: 0.5000
Epoch 5/100
586/586 [==============================] - 15s 25ms/step - loss: 0.6933 - accuracy: 0.5024
Epoch 6/100
586/586 [==============================] - 14s 25ms/step - loss: 0.6932 - accuracy: 0.4992
Epoch 7/100
586/586 [==============================] - 14s 25ms/step - loss: 0.6932 - accuracy: 0.5025
Epoch 8/100
586/586 [==============================] - 14s 25ms/step - loss: 0.6931 - accuracy: 0.5007
Epoch 9/100
586/586 [==============================] - 14s 25ms/step - loss: 0.6931 - accuracy: 0.5025
Epoch 10/100
586/586 [==============================] - 14s 25ms/step - l

In [14]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [15]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
display_metrics(y_test,y_pred)

0.49256
0.33001018384520553
0.24628
0.5


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
